# C6 Data loading, storage, and file formats

In [17]:
import numpy as np
import pandas as pd
import os,sys
import matplotlib.pyplot as plt

In [3]:
%pwd   # 输出当前目录

'C:\\workspace'

In [2]:
%cd c:\workspace\pydata

c:\workspace\pydata


## 6.1 Reading and Writing Data in Text Format
## 6.1 读写文本格式的数据

In [11]:
!type ch06\ex1.csv     #"!type" 是在windows中的用法，在UNIX中，要用"!cat"

a,b,c,d,message
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


In [13]:
df = pd.read_csv('ch06\ex1.csv')    #默认分割符为 逗号  默认文件有标题行
df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [14]:
pd.read_table('ch06\ex1.csv', sep= ',')   # read_table的默认分割符为'制表符\t'，这里指定了分隔符

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [15]:
# 当文件没有标题行时
!type ch06\ex2.csv

1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


In [17]:
#让pandas为其分配默认的列名
pd.read_csv('ch06\ex2.csv', header = None)

,0,1,2,3,4
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [19]:
# 自己定义列名
pd.read_csv('ch06\ex2.csv', names = ['a', 'b', 'c', 'd', 'e'])

,a,b,c,d,e
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [20]:
# 需要将其中一列作为索引列
pd.read_csv('ch06\ex2.csv', names = ['a', 'b', 'c', 'd', 'e'], index_col = 'e')

,a,b,c,d
e,,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


In [21]:
# 多列层次化索引
!type ch06\csv_mindex.csv

key1,key2,value1,value2
one,a,1,2
one,b,3,4
one,c,5,6
one,d,7,8
two,a,9,10
two,b,11,12
two,c,13,14
two,d,15,16


In [24]:
parsed = pd.read_csv('ch06\csv_mindex.csv', index_col = ['key1', 'key2'])
parsed

value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  a          9      10
     b         11      12
     c         13      14
     d         15      16

In [25]:
parsed1 = pd.read_csv('ch06\csv_mindex.csv', index_col = ['key2', 'key1'])
parsed1

,,value1,value2
key2,key1,,
a,one,1,2
b,one,3,4
c,one,5,6
d,one,7,8
a,two,9,10
b,two,11,12
c,two,13,14
d,two,15,16


- 我将需要层析话索引的索引列进行了调换，并未出现原来的效果，估计是第一列需要进行排序

### 6.1.3 非固定分割符分割字段的
编写正则表达式作为read_table的分割符

In [26]:
list(open('ch06\ex3.txt'))

['            A         B         C\n',
 'aaa -0.264438 -1.026059 -0.619500\n',
 'bbb  0.927272  0.302904 -0.032399\n',
 'ccc -0.264273 -0.386314 -0.217601\n',
 'ddd -0.871858 -0.348382  1.100491\n']

In [29]:
# 此文件各个字段由数量不定的空白符分割，可以使用正则表达式 \s+ 表示
result = pd.read_table('ch06\ex3.csv', sep = '\s+')
result
#这里由于列名的数量比列的数量少1，故read_table推断第一列是DataFrame的索引

,A,B,C
aaa,-0.264438,-1.026059,-0.619500
bbb,0.927272,0.302904,-0.032399
ccc,-0.264273,-0.386314,-0.217601
ddd,-0.871858,-0.348382,1.100491


### 6.1.4 跳过行、缺失值及其他参数

In [31]:
!type ch06\ex4.csv

# hey!
a,b,c,d,message
# just wanted to make things more difficult for you
# who reads CSV files with computers, anyway?
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


In [37]:
pd.read_csv('ch06\ex4.csv', skiprows = [0, 2, 3])  #跳行 skiprows

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [38]:
# 缺失值  空字符 某个标记值    na_values可以结束欧一组用于表示缺失值的字符串
result2 = pd.read_csv('ch06\ex5.csv')
result2

,something,a,b,c,d,message
0,one,1,2,3,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11,12,foo


In [39]:
pd.isnull(result2)

,something,a,b,c,d,message
0,False,False,False,False,False,True
1,False,False,False,True,False,False
2,False,False,False,False,False,False


In [40]:
result3 = pd.read_csv('ch06\ex5.csv', na_values = ['NULL'])
result3

,something,a,b,c,d,message
0,one,1,2,3,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11,12,foo


In [41]:
# 用字典为各列指定不同的NA标记值
sentinels = {'message':['foo', 'NA'], 'something' : ['three']}
pd.read_csv('ch06\ex5.csv', na_values = sentinels)

,something,a,b,c,d,message
0,one,1,2,3,4,NaN
1,two,5,6,NaN,8,world
2,NaN,9,10,11,12,NaN


- 其他参数见书本167页

## 6.2 Reading text files in pieces
## 6.2 逐块读取文本文件

In [43]:
result = pd.read_csv('ch06\ex6.csv', nrows = 5)
result

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q


In [59]:
# 逐块读取文件，设置chunksize(行数)，主要用于迭代
chunker = pd.read_csv('ch06\ex6.csv', chunksize=1000)
chunker

In [60]:
tot = pd.Series([])
for piece in chunker:
    tot = tot.add(piece['key'].value_counts(), fill_value = 0)

tot = tot.order(ascending = False)

C:\Python34\lib\site-packages\ipykernel\__main__.py:5: FutureWarning: order is deprecated, use sort_values(...)


In [61]:
tot[:10]

E    368
X    364
L    346
O    343
Q    340
M    338
J    337
F    335
K    334
H    330
dtype: float64

- 书中说TextParser还有一个get_chunk方法，可以读取任意大小的块。但是并不了解这个

## 6.3 Writing data out to text format
## 6.3 将数据写到文本格式

In [62]:
data = pd.read_csv('ch06\ex5.csv')
data

,something,a,b,c,d,message
0,one,1,2,3,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11,12,foo


In [63]:
data.to_csv('ch06\out.csv')
!type ch06\out.csv

,something,a,b,c,d,message
0,one,1,2,3.0,4,
1,two,5,6,,8,world
2,three,9,10,11.0,12,foo


In [68]:
data.to_csv(sys.stdout, sep = '|') # 直接写到sys.stdout仅仅是打印出文本

|something|a|b|c|d|message
0|one|1|2|3.0|4|
1|two|5|6||8|world
2|three|9|10|11.0|12|foo


In [69]:
# 缺失值在输出 结果中会被表示为空字符串，可以表示为其他值
data.to_csv(sys.stdout, na_rep = 'NULL')

,something,a,b,c,d,message
0,one,1,2,3.0,4,NULL
1,two,5,6,NULL,8,world
2,three,9,10,11.0,12,foo


In [71]:
# 禁用列和行标
data.to_csv(sys.stdout, index = False, header = False)

one,1,2,3.0,4,
two,5,6,,8,world
three,9,10,11.0,12,foo


Series的to_csv方法和from_csv

## 6.4 手工处理分隔符格式
## 6.4 Manually working whith delimited formats

In [3]:
!type ch06\ex7.csv

"a","b","c"
"1","2","3"
"1","2","3","4"


In [4]:
import csv
f = open('ch06\ex7.csv')

reader = csv.reader(f)

In [6]:
for line in reader:
    print(line)

['a', 'b', 'c']
['1', '2', '3']
['1', '2', '3', '4']


In [9]:
#数据整理
lines = list(csv.reader(open('ch06\ex7.csv')))
header , values = lines[0], lines[1:]
data_dict = {h: v for h,v in zip(header, zip(*values))}
data_dict

{'a': ('1', '1'), 'b': ('2', '2'), 'c': ('3', '3')}

In [10]:
# CSV文件的形式有很多，可定义csv.Dialect的一个子类即可定义出新格式
class my_dialect(csv.Dialect):
    lineterminator = '\n'
    delimiter = ';'
    quotechar = '"'
    
#这句和后面的一些关于csv的就不太理解，感觉没有简单，反而复杂 了

## 6.5 JSON数据
## 6.5 JSON data

In [11]:
obj = """
{"name": "Wes",
 "places_lived": ["United States", "Spain", "Germany"],
 "pet": null,
 "siblings": [{"name": "Scott", "age": 25, "pet": "Zuko"},
              {"name": "Katie", "age": 33, "pet": "Cisco"}]
}
"""

In [12]:
import json
result = json.loads(obj)
result

{'name': 'Wes',
 'pet': None,
 'places_lived': ['United States', 'Spain', 'Germany'],
 'siblings': [{'age': 25, 'name': 'Scott', 'pet': 'Zuko'},
  {'age': 33, 'name': 'Katie', 'pet': 'Cisco'}]}

In [14]:
# json.loads 将JSON字符串转化为Python形式
# json.dumps 将Python对象转换为JSON格式
asjson = json.dumps(result)
asjson

'{"places_lived": ["United States", "Spain", "Germany"], "siblings": [{"age": 25, "pet": "Zuko", "name": "Scott"}, {"age": 33, "pet": "Cisco", "name": "Katie"}], "pet": null, "name": "Wes"}'

In [22]:
# 将JSON格式转换为DataFrame格式
siblings = pd.DataFrame(result['siblings'], columns = ['name', 'age'])
siblings
#但是这个还是先通过转换为Python原始格式再到DataFrame中的

,name,age
0,Scott,25
1,Katie,33


In [25]:
# to_json from_json   read_json

## 6.6 XML和HTML：Web信息收集
## 6.6 XML and HTML: Web scraping

In [26]:
from lxml.html import parse
from urllib.request import urlopen

In [29]:
# 首先找到希望获得数据的URl，利用urllib.request.urlopen打开，然后用lxml解析得到的数据流
parsed = parse(urlopen('http://finance.yahoo.com/q/op?s=AAPL+Options'))
doc= parsed.getroot()

<Element html at 0x11139a8>

In [30]:
# 通过这个对象，可以获取特定类型的所有HTML标签tag，
links = doc.findall('.//a')
links[10:15]

[<Element a at 0x1113f48>,
 <Element a at 0x1113f98>,
 <Element a at 0x111f048>,
 <Element a at 0x111f098>,
 <Element a at 0x111f0e8>]

In [31]:
lnk = links[15]
lnk.get('href')

'https://www.yahoo.com/movies'

In [32]:
lnk.text_content()

'Movies'

In [34]:
urls = [lnk.get('href') for lnk in doc.findall('.//a')]
urls[5:15]

['http://finance.yahoo.com/',
 'https://weather.yahoo.com/',
 'https://games.yahoo.com/',
 'https://answers.yahoo.com/',
 'https://screen.yahoo.com/',
 'https://www.flickr.com/',
 'https://mobile.yahoo.com/',
 'http://everything.yahoo.com/',
 'https://www.yahoo.com/politics',
 'https://celebrity.yahoo.com/']

In [37]:
tables = doc.findall('.//table')
calls = tables[1]

## 6.8 使用数据库
## 6.8 Interacting with databases
中间几节暂时忽略

In [38]:
import sqlite3

query = """
CREATE TABLE test
(a VARCHAR(20), b VARCHAR(20),c REAL,        d INTEGER
);"""

con = sqlite3.connect(':memory:')
con.execute(query)
con.commit()

data = [('Atlanta', 'Georgia', 1.25, 6),
        ('Tallahassee', 'Florida', 2.6, 3),
        ('Sacramento', 'California', 1.7, 5)]
stmt = "INSERT INTO test VALUES(?, ?, ?, ?)"

con.executemany(stmt, data)
con.commit()

cursor = con.execute('select * from test')
rows = cursor.fetchall()
rows

[('Atlanta', 'Georgia', 1.25, 6),
 ('Tallahassee', 'Florida', 2.6, 3),
 ('Sacramento', 'California', 1.7, 5)]

In [45]:
# 将这个元组列表传给DataFrame的构造器，但是需要位于游标的decription属性中的列名
cursor.description

(('a', None, None, None, None, None, None),
 ('b', None, None, None, None, None, None),
 ('c', None, None, None, None, None, None),
 ('d', None, None, None, None, None, None))

In [46]:
import pandas.io.sql as sql
sql.read_sql('select * from test', con)

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5
